## <center> **Credit Card Fraud (Anomaly/Outlier) Detection** 
<center><em>The processes of determining an entry among entries that does not seem to belong. In this case we are using the Anonymized Credit Card dataset from Kaggle, which has  transactions labeled as fraudulent or genuine to create an ML Fraud Detection model.</em></center>
<br>
<center><img src="https://github.com/kkrusere/Credit-Card-Fraud-Anomaly-Outlier-Detection/blob/main/Assets/CCFD.png" width=600/></center>

***Project Contributors:*** Kuzi Rusere<br>
**MBA streamlit App URL:** N/A

Anomaly detection (also referred to as outlier detection and sometimes as novelty detection) is generally understood to be the identification of rare items, events, or observations that deviate significantly from the majority of the data and do not conform to a well-defined notion of normal behavior.  **Supervised anomaly detection** techniques require a data set that has been labeled as "normal" and "abnormal" and involves training a classifier. **Unsupervised anomaly detection** techniques assume the data is unlabelled and are by far the most commonly used due to their wider and relevant application. **Semi-supervised anomaly detection** techniques assume that some portion of the data is labeled. 

* An anomaly is a point or collection of points that are relatively distant from other points in a multi-dimensional space of features.
* An outlier is an observation (or subset of observations) that appears to be inconsistent with the remainder of that set of data.

There are multiple methods/approaches to undertaking anomaly (or outlier) detection and these are sort-of classified into 3:
* **Statistical**, 
>

* (General/Classical) **Machine Learning**, 
>

* **Deep Learning** 
>

